## Simple Gen AI App using LangChain

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")

#Langsmith Tracking
os.environ["LANGSMITH_API_KEY"]=os.getenv("LANGSMITH_API_KEY")
os.environ["LANGSMITH_TRACING"]="true"
os.environ["LANGSMITH_PROJECT"]=os.getenv("LANGSMITH_PROJECT")

In [2]:
## Data Ingestion --From the website  we need yo scrape the data
from langchain_community.document_loaders import WebBaseLoader

c:\Users\SAM\Documents\anjali\Udemy-Agentic AI\Langchain Project\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
loader=WebBaseLoader("https://docs.langchain.com/oss/python/langchain/agents")
loader

In [4]:
docs=loader.load()
docs


[Document(metadata={'source': 'https://docs.langchain.com/oss/python/langchain/agents', 'title': 'Agents - Docs by LangChain', 'language': 'en'}, page_content='Agents - Docs by LangChainSkip to main contentDocs by LangChain home pageLangChain + LangGraphSearch...⌘KAsk AIGitHubTry LangSmithTry LangSmithSearch...NavigationCore componentsAgentsLangChainLangGraphDeep AgentsIntegrationsLearnReferenceContributePythonOverviewGet startedInstallQuickstartChangelogPhilosophyCore componentsAgentsModelsMessagesToolsShort-term memoryStreamingStructured outputMiddlewareOverviewBuilt-in middlewareCustom middlewareAdvanced usageGuardrailsRuntimeContext engineeringModel Context Protocol (MCP)Human-in-the-loopMulti-agentRetrievalLong-term memoryAgent developmentLangSmith StudioTestAgent Chat UIDeploy with LangSmithDeploymentObservabilityOn this pageCore componentsModelStatic modelDynamic modelToolsDefining toolsTool error handlingTool use in the ReAct loopDynamic toolsSystem promptDynamic system promptI

In [5]:
### Load Data--> Docs--> Divide our text into chunks ---> text -->vectors
#  using Vector Embedding(techninques convert text into vectors)
# --> Vector store DB
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs) #chunks of my docs

 


In [6]:
documents

[Document(metadata={'source': 'https://docs.langchain.com/oss/python/langchain/agents', 'title': 'Agents - Docs by LangChain', 'language': 'en'}, page_content='Agents - Docs by LangChainSkip to main contentDocs by LangChain home pageLangChain + LangGraphSearch...⌘KAsk AIGitHubTry LangSmithTry LangSmithSearch...NavigationCore componentsAgentsLangChainLangGraphDeep AgentsIntegrationsLearnReferenceContributePythonOverviewGet startedInstallQuickstartChangelogPhilosophyCore componentsAgentsModelsMessagesToolsShort-term memoryStreamingStructured outputMiddlewareOverviewBuilt-in middlewareCustom middlewareAdvanced usageGuardrailsRuntimeContext engineeringModel Context Protocol (MCP)Human-in-the-loopMulti-agentRetrievalLong-term memoryAgent developmentLangSmith StudioTestAgent Chat UIDeploy with LangSmithDeploymentObservabilityOn this pageCore componentsModelStatic modelDynamic modelToolsDefining toolsTool error handlingTool use in the ReAct loopDynamic toolsSystem promptDynamic system promptI

In [7]:
## Converting into vectors using openai

In [8]:
from langchain_openai import OpenAIEmbeddings
embeddings=OpenAIEmbeddings()

In [9]:
from langchain_community.vectorstores import FAISS
vectorstoredb=FAISS.from_documents(documents,embeddings)

In [10]:
vectorstoredb

In [11]:
#QUERY FROM A VECTOR DB
query="what is provider strategy"
result=vectorstoredb.similarity_search(query)
result[0].page_content

'promptDynamic system promptInvocationAdvanced conceptsStructured outputToolStrategyProviderStrategyMemoryDefining state via middlewareDefining state via state_schemaStreamingMiddlewareCore componentsAgentsCopy pageCopy pageAgents combine language models with tools to create systems that can reason about tasks, decide which tools to use, and iteratively work towards solutions.'

In [12]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o")

Custom prompt template

In [23]:
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


prompt = ChatPromptTemplate.from_template(
    """
    Answer the following question based only on the provided context.

    <context>
    {context}
    </context>

    """
)

document_chain = create_stuff_documents_chain(llm, prompt)

In [24]:
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n    Answer the following question based only on the provided context.\n\n    <context>\n    {context}\n    </context>\n\n    '), additional_kwargs={})])
| ChatOpenAI(profile={'max_input_tokens': 128000, 'max_output_tokens': 16384, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'pdf_inputs': True, 'pdf_tool_message': True, 'image_tool_message': True, 'tool_choice': True}, client=<openai.resources

In [25]:
from langchain_core.documents import Document
document_chain.invoke({
    "input":"create_agent provides a production-ready agent implementation.",
    "context":[Document(page_content="Agents combine language models with tools to create systems that can reason about tasks, decide which tools to use, and iteratively work towards solutions.")]
})

'The provided context explains that agents use language models in conjunction with tools to build systems capable of reasoning about tasks, selecting appropriate tools, and progressively working toward solutions.'

In [ ]:
### Input --->Retreiver(Interface)-->VectorStoreDb
vectorstoredb

In [ ]:
retriever=vectorstoredb.as_retriever() # from vectorstoredb

from langchain_classic.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever,document_chain) #document_chain gives context info


In [29]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002559947F2B0>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n    Answer the following question based only on the provided context.\n\n    <context>\n    {context}\n    </context>\n\n    '), additional_kwargs=

In [30]:
### Get the response from the llm 
response=retrieval_chain.invoke({"input":"create_agent provides a production-ready agent implementation."})

In [31]:
response

{'input': 'create_agent provides a production-ready agent implementation.',
 'context': [Document(id='16098585-0723-444f-bb5a-30abc66d7e8b', metadata={'source': 'https://docs.langchain.com/oss/python/langchain/agents', 'title': 'Agents - Docs by LangChain', 'language': 'en'}, page_content='create_agent provides a production-ready agent implementation.\nAn LLM Agent runs tools in a loop to achieve a goal.\nAn agent runs until a stop condition is met - i.e., when the model emits a final output or an iteration limit is reached.'),
  Document(id='985dd908-7ff6-4a14-9805-650961f5f699', metadata={'source': 'https://docs.langchain.com/oss/python/langchain/agents', 'title': 'Agents - Docs by LangChain', 'language': 'en'}, page_content='create_agent builds a graph-based agent runtime using LangGraph. A graph consists of nodes (steps) and edges (connections) that define how your agent processes information. The agent moves through this graph, executing nodes like the model node (which calls the 

In [35]:
response['context']

[Document(id='16098585-0723-444f-bb5a-30abc66d7e8b', metadata={'source': 'https://docs.langchain.com/oss/python/langchain/agents', 'title': 'Agents - Docs by LangChain', 'language': 'en'}, page_content='create_agent provides a production-ready agent implementation.\nAn LLM Agent runs tools in a loop to achieve a goal.\nAn agent runs until a stop condition is met - i.e., when the model emits a final output or an iteration limit is reached.'),
 Document(id='985dd908-7ff6-4a14-9805-650961f5f699', metadata={'source': 'https://docs.langchain.com/oss/python/langchain/agents', 'title': 'Agents - Docs by LangChain', 'language': 'en'}, page_content='create_agent builds a graph-based agent runtime using LangGraph. A graph consists of nodes (steps) and edges (connections) that define how your agent processes information. The agent moves through this graph, executing nodes like the model node (which calls the model), the tools node (which executes tools), or middleware.Learn more about the Graph A